<a href="https://colab.research.google.com/github/FelipeHondei/ServidorLap/blob/main/Automa%C3%A7%C3%A3o_CIGAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Formatar o arquivo**

In [ ]:
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import re
import codecs

# Limpar uma linha do arquivo
def clean_line(line):
    pattern = re.compile(r'[^\x00-\x7F]')  # Encontrar caracteres inválidos
    return pattern.sub(' ', line)  # Substitui caracteres inválidos por espaço em branco

# Definindo o caminho do arquivo
file_path = "/content/drive/MyDrive/VENDVEIM02.csv"  #caminho do arquivo

# Abre o arquivo original para leitura e um novo arquivo para escrita
with codecs.open(file_path, 'r', encoding='utf-8', errors='ignore') as infile, \
     open('cleaned_file.csv', 'w', encoding='utf-8') as outfile:

    for line in infile:
        cleaned_line = clean_line(line)
        outfile.write(cleaned_line)

# Tenta ler o arquivo limpo com Pandas
df = pd.read_csv('cleaned_file.csv', dtype=str, encoding='utf-8', sep=';', index_col=False)  # Certifique-se do delimitador

# Substituir valores na coluna "Branch"
branch_map = {"01": "9117", "02": "9118", "03": "9119", "04": "9120",
              "05": "9121", "06": "9122", "07": "9421", "08": "9431"}
df["Branch"] = df["Branch"].map(branch_map)

coluna_ad = "Invoice date"
coluna_af = "Chassis"
coluna_a = "Branch"

df[coluna_af] = pd.to_numeric(df[coluna_af], errors='coerce')
df.sort_values(by=coluna_af, ascending=True, inplace=True)  # Ordenar de menor para maior

colunas = [coluna_a, coluna_ad, coluna_af] + [col for col in df.columns if col not in [coluna_a, coluna_ad, coluna_af]]
df = df[colunas]

indice_coluna_remover = df.columns.get_loc('Dealer Number')  # Obtém o índice da coluna pelo nome
df = df.drop(df.columns[indice_coluna_remover], axis=1)

# Nome do arquivo de saída
data_hoje = datetime.today().strftime('%d-%m-%Y')
output_file = f"Faturamento ({data_hoje}).xlsx"

# Salvar em Excel
df.to_excel(output_file, index=False)

# Aplicar formatação verde ao cabeçalho
wb = load_workbook(output_file)
ws = wb.active
fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
for cell in ws[1]:
    cell.fill = fill
wb.save(output_file)

print(f"Arquivo salvo como {output_file}")


# Envia por email

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from datetime import datetime
from email.mime.text import MIMEText
import os

# enviar e-mail com anexo
def send_email_with_attachment(subject, body, to_emails, attachment_path):
    # Configurações do e-mail
    from_email = "Email@usuario.com.br"  # Login email
    from_password = "Senha"  # Senha do email


    # Criando o e-mail
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['Subject'] = subject
    msg['To'] = ", ".join(to_emails)  # Quem vai receber o email

    body = "Prezado(a),\n\nSegue em anexo o relatório de faturamento do dia."

    # Criando a parte do corpo do email
    body_part = MIMEText(body, 'plain')

    # Corpo do e-mail
    msg.attach(body_part)

    # Anexando o arquivo
    with open(attachment_path, 'rb') as file:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(file.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f"attachment; filename={os.path.basename(attachment_path)}")
        msg.attach(part)

    # Enviando o e-mail
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(from_email, from_password)
        server.sendmail(from_email, to_emails, msg.as_string())
        server.quit()
        print("E-mail enviado com sucesso!")
    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Nome do arquivo de saída (gerado previamente)
data_hoje = datetime.today().strftime('%d-%m-%Y')
output_file = f"Faturamento ({data_hoje}).xlsx"

# Detalhes do e-mail
subject = "Relatório de Faturamento"


# Lista de destinatários (adicione os e-mails que você deseja enviar)
to_emails = ["felipe.hondei@laponia.com.br"] #Coloquei meu email para teste

# Enviar e-mail com anexo
send_email_with_attachment(subject, body, to_emails, output_file)

# **Baixar o arquivo**
(Não testei pois não consegui a conexão por email)

In [ ]:
import imaplib
import email
import os

EMAIL = "email@laponia.com"
SENHA = "senha"  # Acho que vai precisar da senha "aplicativo"
IMAP_SERVER = "outlook.office365.com"
PASTA_DESTINO = "downloads"
REMETENTES_PERMITIDOS = ["email1@laponia.com", "email2@laponia.com"]

# Criar pasta para salvar os arquivos
if not os.path.exists(PASTA_DESTINO):
    os.makedirs(PASTA_DESTINO)

try:
    # Servidor IMAP
    mail = imaplib.IMAP4_SSL(IMAP_SERVER)
    mail.login(EMAIL, SENHA)
    mail.select("inbox")

    # Buscar e-mails de remetentes específicos com anexos
    status, mensagens = mail.search(None, "ALL")
    for num in mensagens[0].split():
        status, dados = mail.fetch(num, "(RFC822)")
        raw_email = dados[0][1]

        # Ler e analisar o e-mail para achar o email certo para download do arquivo
        msg = email.message_from_bytes(raw_email)
        remetente = msg["From"]

        if any(remetente.lower().find(r.lower()) != -1 for r in REMETENTES_PERMITIDOS):
            for part in msg.walk():
                if part.get_content_disposition() == "attachment":
                    filename = part.get_filename()
                    if filename and filename.endswith(".csv"):
                        filepath = os.path.join(PASTA_DESTINO, filename)
                        with open(filepath, "wb") as f:
                            f.write(part.get_payload(decode=True))
                        print(f"Arquivo baixado: {filename}")

    # Fechar conexão
    mail.logout()

except Exception as e:
    print(f"Erro: {e}")
